<a href="https://colab.research.google.com/github/Nada-gh/Web-Scraping-with-Selenium/blob/master/Airbnb_Web_ScrapingColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (71.0.3578.98-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
cp: cannot create regular file '/usr/bin/chromedriver': Text file busy


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [0]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException 
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.keys import Keys #need to send keystrokes
from datetime import datetime 
from datetime import timedelta
from dateutil import parser
import urllib
from urllib.parse import urlparse
from urllib.parse import parse_qs
import csv
import time

In [25]:
# create new instance of chrome 
#browser = webdriver.Chrome(executable_path='C:/webdriver/chromedriver')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

start_date = '2019-03-06'
start_date = parser.parse(start_date)

SCROLL_PAUSE_TIME = 0.5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [26]:
with open('AirBnbOneDayColab.csv','w',encoding="utf-8",newline='') as csvfile:
    fieldnames = ['Date','Has_Weekend','Title','Price','Super_host','Latitude','Longitude','EntireHomeVsRoom','Bedrooms_no','Beds_no','Bathrooms_num','Sleeping_engagment','Amenities','Accuracy_stars','Communication_stars','Cleanliness_stars','Location_stars','CheckIn_stars','Value_stars','Reviewers_no','Review']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    # how many days to loop through
    for h in range(1,2):
        Checkin_Date = start_date+timedelta(days=1)# check_in =19+1=20
        Checkout_Date = start_date+timedelta(days=6)# check_out = 19+2=21
        browser.get("https://www.airbnb.ca/s/homes?refinement_paths%5B%5D=%2Fhomes&query=New%20York&allow_override%5B%5D=&checkin="+Checkin_Date.strftime('%Y-%m-%d')+"&checkout="+Checkout_Date.strftime('%Y-%m-%d')+"&map_toggle=false")
        browser.maximize_window()
        last_height = browser.execute_script("return document.body.scrollHeight")# Get scroll height
        then = time.time()
        while True:
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")# Scroll down to bottom
            time.sleep(SCROLL_PAUSE_TIME) # Wait to load page
            new_height = browser.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
            if new_height == last_height or time.time()-then > 60:
                break
        last_height = new_height
        cont = browser.find_element_by_xpath("//body[@class='with-new-header has-epcot-header']")
        prods = cont.find_elements_by_xpath("//meta[@itemprop='url']")
        links = [prod.get_attribute("content") for prod in prods]  
        for l in links: # 18 or 30 based on the num of the homes in the pag
            driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
            driver.get("https://"+l)
            #driver.maximize_window()
            tot = driver.find_element_by_xpath("//body[@class='with-new-header has-epcot-header']")

            try:
                title = tot.find_element_by_class_name('_18hrqvin').text
            except NoSuchElementException:
                title = None    
            #try:
            #elem = wait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='_59m2yxn']/img")))
            map_url = driver.find_element_by_xpath("//*[@id='neighborhood']//div[@class='_59m2yxn']/img").get_attribute("src")
            query = parse_qs(urlparse(map_url).query)['center'][0]
            lat = query.split(",")[0]
            long = query.split(",")[1]
            #except NoSuchElementException:
            #lat = None
            #long = None
                
            try:
                for i in range(0,5):
                    day_no = Checkin_Date+timedelta(days=i)
                    if(day_no.weekday()< 5):
                        has_weekend = False
                    else:
                        has_weekend = True
                        break
                Has_weekend = has_weekend
            except NoSuchElementException:
                Has_weekend = None    

            try:
                price = tot.find_element_by_class_name('_doc79r').text
            except NoSuchElementException:
                price = None
                
            try:
                EntireHomeVsRoom = tot.find_element_by_class_name("_1xxanas2").text
            except NoSuchElementException:
                EntireHomeVsRoom = None
                
            #beds_bedrooms= tot.find_elements_by_xpath("//div[@id='room']//*[contains(text(),'bed')]")
                
            try:  
                num_bedrooms = tot.find_element_by_xpath("//div[@id='room']//*[contains(text(),'bedroom')]").text
            except NoSuchElementException:
                num_bedrooms = None
                                                    
            try:                                          
                num_beds = tot.find_element_by_xpath("//*[contains(text(),'bed')][not(contains(text(), 'bedroom'))]").text
            except NoSuchElementException:
                num_beds = None
            
            try:
                num_baths= tot.find_element_by_xpath("//*[@class='_czm8crp' and contains(text(),'bath')]").text
            except NoSuchElementException:
                num_baths = None
                                                   
            try:
                sleeping_engagments = tot.find_element_by_xpath("//div[@class='_520jgb']").text
            except NoSuchElementException:
                sleeping_engagments = None 
                        
            try:
                open_amenities_window = tot.find_element_by_xpath("//button[@class='_pgfpyhv' and contains(text(),'amenities')]").send_keys("\n")
                amenities= tot.find_elements_by_xpath("//div[@aria-labelledby='dls-modal__AmenitiesModal']//div[@class='_czm8crp']")
                amenities_total=[amenities[i].text for i in range(0,len(amenities))] 
            except NoSuchElementException:
                amenities_total= None
            #close_amenities_window = tot.find_element_by_xpath("//div[@class='_152qbzi']/button[@class='_1rp5252' and @aria-label='Close']").send_keys("\n")    
            
            time.sleep(3)   
            driver.refresh()
                                                        
            try:
                reviews_group = driver.find_elements_by_xpath("//*[@class='_1iu38l3']/span")
                Accuracy_stars = reviews_group[0].get_attribute("aria-label")
                Communication_stars = reviews_group[1].get_attribute("aria-label")
                Cleanliness_stars = reviews_group[2].get_attribute("aria-label")
                Location_stars = reviews_group[3].get_attribute("aria-label")
                CheckIn_stars = reviews_group[4].get_attribute("aria-label")
                Value_stars = reviews_group[5].get_attribute("aria-label")
            except:
                Accuracy_stars = None
                Communication_stars = None
                Cleanliness_stars = None
                Location_stars = None
                CheckIn_stars = None
                Value_stars = None
                pass 
                                                                   
            try:
                review_txt = driver.find_element_by_xpath("//div[@id='reviews']").text
            except (NoSuchElementException,StaleElementReferenceException) as e:
                review_txt = None
            
            try:
                Super_host = driver.find_element_by_xpath("//*[@class='_1p3joamp' and contains(text(),'Superhos')]").text
            except (NoSuchElementException,StaleElementReferenceException) as e:
                Super_host = None
                                                      
            try:                                         
                reviewers_num = tot.find_element_by_xpath("//button[@class='_ff6jfq']//span[@class='_so3dpm2']").text
            except (NoSuchElementException,StaleElementReferenceException) as e:
                reviewers_num = None
                                                      

            writer.writerow({'Date' : Checkin_Date,
                             'Title': title,
                             'Price': price,
                             'Has_Weekend' : Has_weekend,
                             'Latitude' : lat,
                             'Longitude' : long,
                             'EntireHomeVsRoom' : EntireHomeVsRoom,
                             'Bedrooms_no': num_bedrooms,
                             'Beds_no': num_beds,
                             'Bathrooms_num' : num_baths,
                             'Sleeping_engagment':sleeping_engagments,
                             'Amenities': amenities_total,
                             'Accuracy_stars': Accuracy_stars,
                             'Communication_stars': Communication_stars,
                             'Cleanliness_stars' : Cleanliness_stars,
                             'Location_stars' : Location_stars,
                             'CheckIn_stars' : CheckIn_stars,
                             'Value_stars' : Value_stars,
                             'Reviewers_no': reviewers_num,
                             'Super_host': Super_host,
                             'Review':review_txt})

            time.sleep(10)
            driver.close()
            
        start_date = Checkin_Date        
                
               # browser.find_element_by_xpath("//li[@class='_pgfpyhv']/a").send_keys("\n") #send enter for links, buttons
            #inputElement.send_keys("\n") #send enter for links, buttons
         # Calculate new scroll height and compare with last scroll height
        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/p